Import libraries used for this shit.

In [4]:
# !pip install datasets evaluate --upgrade

You should consider upgrading via the 'C:\Users\Sy Quan\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import re
import datasets

from collections import Counter, OrderedDict
from typing import Dict, Iterable, List, Optional

c:\Users\Sy Quan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Just in case we need to use datasets' utilities

In [6]:
# dataset = datasets.load_dataset('csv', data_files={'train': 'data/train.csv', 'test': 'data/test.csv', 'valid': 'data/valid.csv'})

In [34]:
# train_datas, valid_datas, test_datas = (
#     dataset["train"],
#     dataset["valid"],
#     dataset["test"],
# )

Load the dataset.

In [8]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
valid_data = pd.read_csv('data/valid.csv')

Only take data from source: 'OpenSubtitles v2018'.

In [35]:
# train_datas = train_datas.filter(lambda example: example['source'] == 'OpenSubtitles v2018')

In [51]:
train_data = train_data[train_data['source'] == 'OpenSubtitles v2018']
test_data = test_data[test_data['source'] == 'OpenSubtitles v2018']
valid_data = valid_data[valid_data['source'] == 'OpenSubtitles v2018']

In [36]:
# train_datas = train_datas.shuffle(seed=42).select(range(50))

In [56]:
train_data_sampled = train_data.sample(n=50, random_state=42)

Take the first 2 columns from the dataset (english and vietnamese, respectively).

In [57]:
train_en = train_data_sampled['en']
train_vi = train_data_sampled['vi']

test_en = test_data['en']
test_vi = test_data['vi']

valid_en = valid_data['en']
valid_vi = valid_data['vi']

Just checking.

In [39]:
# train_datas

Dataset({
    features: ['en', 'vi', 'source'],
    num_rows: 50
})

In [58]:
print(train_en.shape)

(50,)


In [59]:
train_en.head()

2269779                       There are thousands of glens.
2674860                               I pushed you in that.
2541945                 So he wouldn't lie or use an alias.
1865205    I remember identifying you after the exhumation.
831567              This is where you've been for 20 years?
Name: en, dtype: object

## Tokenizers

A tokenizer is used to turn a string into a list of tokens that make up that string, e.g. "good morning!" becomes ["good", "morning", "!"]. We'll start talking about the sentences being a sequence of tokens from now, instead of saying they're a sequence of words. What's the difference? Well, "good" and "morning" are both words and tokens, but "!" is not a word. We could say "!" is punctuation, but the term token is more general and covers: words, punctuation, numbers and any special symbols.

This function helps with special suffices.

In [60]:
def split_suffix(word):
    # Check for "n't" suffix (negative contractions)
    if re.search(r"n't$", word):
        return [word]  # Return the whole word as a single token
    
    # Check for specific suffixes ('d, 've, 's, 'm, 'll, 're)
    match = re.search(r"(.*)('d|'ve|'s|'m|'ll|'re)$", word)
    if match:
        return [match.group(1), match.group(2)]  # Return the part before and the suffix

    # Check for a word ending with just '
    match = re.search(r"(.*)('$)$", word)
    if match:
        return [match.group(1), match.group(2)]  # Return the part before and just '

    # Check for apostrophe + other characters (return <unk>)
    if re.search(r"(.*)('\w+)$", word):
        return ["<unk>"]  # Return <unk>

    return [word]  # Return the word unchanged if no match

The following function tokenizes a line of text.

In [61]:
patterns = [r"\"", r"\.", r"<br \/>", r",", r"\(", r"\)", r"\!", r"\?", r"\;", r"\:", r"\s+"]
replacements = ["", " . ", " ", " , ", " ( ", " ) ", " ! ", " ? ", " ", " ", " "]
patterns_dict = list((re.compile(p), r) for p, r in zip(patterns, replacements))

def get_tokenizer_line(line):
    """
    Basic normalization for a line of text.
    Normalization includes
    - lowercasing
    - complete some basic text normalization for English words as follows:
        add spaces before and after '\''
        remove '\"',
        add spaces before and after '.'
        replace '<br \/>'with single space
        add spaces before and after ','
        add spaces before and after '('
        add spaces before and after ')'
        add spaces before and after '!'
        add spaces before and after '?'
        replace ';' with single space
        replace ':' with single space
        replace multiple spaces with single space

    Returns a list of tokens after splitting on whitespace.
    """
    line = line.lower()
    for pattern_re, replaced_str in patterns_dict:
        line = pattern_re.sub(replaced_str, line)
    result = [split_suffix(word) for word in line.split()]
    flattened_result = [token for sublist in result for token in sublist]
    return flattened_result

print(get_tokenizer_line("You can now install TorchText (but I can't) and this's for checking the split_suffix functions' i'dk!"))

['you', 'can', 'now', 'install', 'torchtext', '(', 'but', 'i', "can't", ')', 'and', 'this', "'s", 'for', 'checking', 'the', 'split_suffix', 'functions', "'", '<unk>', '!']


The following function tokenizes text (in general).

In [62]:
def get_tokenizer(text):
    tokens = []
    for line in text:
        tokens.append(get_tokenizer_line(line))
    return tokens
        
get_tokenizer(["You can now install TorchText using pip!", "This is a test."])

[['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!'],
 ['this', 'is', 'a', 'test', '.']]

This function tokenizes the dataset.

In [63]:
def tokenize_example(example, max_length, sos_token, eos_token):
    """
    Apply the tokenizer to all of the examples in each data split, as well as some other processing.

    This function takes in an example from the dataset, applies the get_tokenizer function, trims the list of tokens to a maximum length, 
    and then appends the start of sequence and end of sequence tokens to the beginning and end of the list of tokens.
    """
    en_tokens = get_tokenizer(example["en"])[:max_length]
    vi_tokens = get_tokenizer(example["vi"])[:max_length]
    en_tokens = [[sos_token] + token + [eos_token] for token in en_tokens]
    vi_tokens = [[sos_token] + token + [eos_token] for token in vi_tokens]
    return {"en_tokens": en_tokens, "vi_tokens": vi_tokens}

In [64]:
tokenize_example(train_data_sampled[:5], 20, "<sos>", "<eos>")

{'en_tokens': [['<sos>',
   'there',
   'are',
   'thousands',
   'of',
   'glens',
   '.',
   '<eos>'],
  ['<sos>', 'i', 'pushed', 'you', 'in', 'that', '.', '<eos>'],
  ['<sos>',
   'so',
   'he',
   "wouldn't",
   'lie',
   'or',
   'use',
   'an',
   'alias',
   '.',
   '<eos>'],
  ['<sos>',
   'i',
   'remember',
   'identifying',
   'you',
   'after',
   'the',
   'exhumation',
   '.',
   '<eos>'],
  ['<sos>',
   'this',
   'is',
   'where',
   'you',
   "'ve",
   'been',
   'for',
   '20',
   'years',
   '?',
   '<eos>']],
 'vi_tokens': [['<sos>',
   'có',
   'hàng',
   'ngàn',
   'thung',
   'lũng',
   'như',
   'thế',
   '.',
   '<eos>'],
  ['<sos>', 'tớ', 'đẩy', 'cậu', 'đi', 'bằng', 'cái', 'đó', '.', '<eos>'],
  ['<sos>',
   'nên',
   'hắn',
   'sẽ',
   'không',
   'nói',
   'dối',
   'hay',
   'dùng',
   'bí',
   'danh',
   '.',
   '<eos>'],
  ['<sos>',
   'anh',
   'nhớ',
   'lại',
   'lúc',
   'nhận',
   'dạng',
   'em',
   'khi',
   'họ',
   'đào',
   'mộ',
   '.',
   '<eo

Here, we're trimming all sequences to a maximum length of 1000000000 tokens, and using *sos* and *eos* as the start and end of sequence tokens, respectively.

In [65]:
max_length = 1_000_000_000
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data_tokenized = tokenize_example(train_data_sampled, **fn_kwargs)
valid_data_tokenized = tokenize_example(valid_data, **fn_kwargs)
test_data_tokenized = tokenize_example(test_data, **fn_kwargs)

In [44]:
# max_length = 1_000_000_000
# sos_token = "<sos>"
# eos_token = "<eos>"

# fn_kwargs = {
#     "max_length": max_length,
#     "sos_token": sos_token,
#     "eos_token": eos_token,
# }

# train_datas_tokenized = train_datas.map(tokenize_example, fn_kwargs=fn_kwargs)
# test_datas_tokenized = test_datas.map(tokenize_example, fn_kwargs=fn_kwargs)
# valid_datas_tokenized = valid_datas.map(tokenize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 50/50 [00:00<00:00, 320.14 examples/s]


We can now look at some examples, confirming the two new features have been added; both of which are lowercased list of strings with the start/end of sequence tokens appended.

In [66]:
print(train_data_tokenized)

{'en_tokens': [['<sos>', 'there', 'are', 'thousands', 'of', 'glens', '.', '<eos>'], ['<sos>', 'i', 'pushed', 'you', 'in', 'that', '.', '<eos>'], ['<sos>', 'so', 'he', "wouldn't", 'lie', 'or', 'use', 'an', 'alias', '.', '<eos>'], ['<sos>', 'i', 'remember', 'identifying', 'you', 'after', 'the', 'exhumation', '.', '<eos>'], ['<sos>', 'this', 'is', 'where', 'you', "'ve", 'been', 'for', '20', 'years', '?', '<eos>'], ['<sos>', 'look', ',', 'let', "'s", 'put', 'some', 'real', 'miles', 'on', 'those', 'bikes', '.', '<eos>'], ['<sos>', '-', 'i', "don't", 'know', 'what', 'happened', '.', '<eos>'], ['<sos>', 'comes', 'out', 'to', 'a', 'million', ',', "doesn't", 'it', '?', '<eos>'], ['<sos>', 'it', "'s", 'not', 'good', 'for', 'a', 'son', 'of', 'pharaoh', 'to', 'see', 'his', 'father', 'humbled', 'by', 'a', 'son', 'of', 'slaves', '.', '<eos>'], ['<sos>', 'it', "'s", 'all', 'happening', 'right', 'in', 'front', 'of', 'your', 'eyes', 'over', 'and', 'over', '.', '<eos>'], ['<sos>', 'well', ',', 'they', '

In [45]:
# train_datas_tokenized.select_columns(["en_tokens"])

Dataset({
    features: ['en_tokens'],
    num_rows: 50
})

In [46]:
# print(train_datas_tokenized[0:5])

{'en': ['Lost the game, but not the spirit', "And I did, and it's great.", 'A sign of surrender. And shame.', 'You seemed to be in the middle of a new disaster every time I turned on the news.', "♪ Oh, when you're almost there ♪ ♪ Almost home ♪"], 'vi': ['Thua trận, nhưng không thua ý chí', 'Và anh đã làm được, điều đó thật tuyệt.', 'là dấu hiệu đầu hàng và sự hổ thẹn.', 'Anh trông giống như đang ở tâm của mội đại dịch mỗi lần tôi bật chương trình tin tức.', 'Khi bạn tới gần nơi đó Gần thật gần.'], 'source': ['OpenSubtitles v2018', 'OpenSubtitles v2018', 'OpenSubtitles v2018', 'OpenSubtitles v2018', 'OpenSubtitles v2018'], 'en_tokens': [[['<sos>', 'l', '<eos>'], ['<sos>', 'o', '<eos>'], ['<sos>', 's', '<eos>'], ['<sos>', 't', '<eos>'], ['<sos>', '<eos>'], ['<sos>', 't', '<eos>'], ['<sos>', 'h', '<eos>'], ['<sos>', 'e', '<eos>'], ['<sos>', '<eos>'], ['<sos>', 'g', '<eos>'], ['<sos>', 'a', '<eos>'], ['<sos>', 'm', '<eos>'], ['<sos>', 'e', '<eos>'], ['<sos>', ',', '<eos>'], ['<sos>', '

## Vocabularies

Next, we'll build the vocabulary for the source and target languages. The vocabulary is used to associate each unique token in our dataset with an index (an integer), e.g. "hello" = 1, "world" = 2, "bye" = 3, "hates" = 4, etc. When feeding text data to our model, we convert the string into tokens and then the tokens into numbers using the vocabulary as a look up table, e.g. "hello world" becomes ["hello", "world"] which becomes [1, 2] using the example indices given. We do this as neural networks cannot operate on strings, only numerical values.

In theory, our vocabulary can be large enough to have an index for every unique token in our dataset. However, what happens if a token exists in our validation and test set, but not in our training set? In that case, we replace the token with an "unknown token", denoted by **_unk_**, which is given its own index (usually index zero). All unknown tokens are replaced by **_unk_**, even if the tokens are different, i.e. if the tokens "gilgamesh" and "enkidu" were both not within our vocabulary, then the string "gilgamesh hates enkidu" gets tokenized to ["gilgamesh", "hates", "enkidu"] and then becomes [0, 24, 0] (where "hates" has the index 24).

Ideally, we want our model to be able to handle unknown tokens by learning to use the context around them to make translations. The only way it can learn that is if we also have unknown tokens in the training set. Hence, when creating our vocabularies with build_vocab_from_iterator, we use the "min_freq" to not create an index for tokens which appear less than min_freq times in our training set. In other words, when using the vocabulary, any token which does not appear at least twice in our training set will get replaced by the unknown token index when converting tokens to indices.

We also use the "specials" argument to pass special tokens. These are tokens which we want to add to the vocabulary but do not necessarily appear in our tokenized examples. These special tokens will appear first in the vocabulary. We've already discussed the unk_token, sos_token, and eos_token. The final special token is the pad_token, denoted by **_pad_**.

When inputting sentences into our model, it is more efficient to pass multiple sentences at once (known as a batch), instead of one at a time. The requirement for sentences to be batched together is that they all have to be the same length (in terms of the number of tokens). The majority of our sentences are not the same length, but we can solve this by "padding" (adding **_pad_**> tokens) the tokenized version of each sentence in a batch until they all have equal tokens to the longest sentence in the batch. For example, if we had two sentences: "I love pizza" and "I hate music videos". They would be tokenized to something like: ["i", "love", "pizza"] and ["i", "hate", "music", "videos"]. The first sequence of tokens would then be padded to ["i", "love", "pizza", "**_pad_**"]. Both sequences could then be converted to indexes using the vocabulary.

That was a lot of information, but torchtext can handle all the fuss of building the vocabulary. Unfortunately, I can't import torchtext, so the whole process will have to be built manually (Đm torchtext).

In [67]:
class Vocabulary:
    def __init__(self, max_vocab_size=None, min_freq=1, specials=['<pad>', '<unk>', '<bos>', '<eos>']):
        self.max_vocab_size = max_vocab_size
        self.min_freq = min_freq
        self.specials = specials
        self.token2idx = {}
        self.idx2token = []
        self.counter = Counter()

        # Adding special tokens first
        self.add_special_tokens()

    def add_special_tokens(self):
        for token in self.specials:
            self.add_token(token)

    def add_token(self, token):
        if token not in self.token2idx:
            idx = len(self.idx2token)
            self.token2idx[token] = idx
            self.idx2token.append(token)

    def build_vocab(self, sentences):
        # Count frequencies of all tokens
        for sentence in sentences:
            self.counter.update(sentence)
        
        # Filter out tokens by frequency and limit vocab size
        most_common = self.counter.most_common(self.max_vocab_size) if self.max_vocab_size else self.counter.items()
        for token, freq in most_common:
            if freq >= self.min_freq and token not in self.token2idx:
                self.add_token(token)

    def __len__(self):
        return len(self.idx2token)

    def get_index(self, token):
        return self.token2idx.get(token, self.token2idx['<unk>'])

    def get_token(self, index):
        return self.idx2token[index] if index < len(self.idx2token) else '<unk>'

    def encode(self, sentence):
        return [self.get_index(token) for token in sentence]

    def decode(self, indices):
        return [self.get_token(index) for index in indices]


In [68]:
# Initialize vocabulary
en_vocab = Vocabulary(max_vocab_size=1_000_000_000, min_freq=1)
vi_vocab = Vocabulary(max_vocab_size=1_000_000_000, min_freq=1)

# Build the vocabulary from sentences
en_vocab.build_vocab(train_data_tokenized["en_tokens"])
vi_vocab.build_vocab(train_data_tokenized["vi_tokens"])

print((len(en_vocab), len(vi_vocab)))


(283, 307)


In [69]:
# Encode a sentence
encoded_sentence = en_vocab.encode(['<bos>', "i", "love", "learning", "and", "i", "am", "gay", "!", ".", "<eos>"])
print("Encoded:", encoded_sentence)

# Decode the sentence
decoded_sentence = en_vocab.decode(encoded_sentence)
print("Decoded:", decoded_sentence)


Encoded: [2, 14, 1, 1, 13, 14, 1, 1, 1, 5, 3]
Decoded: ['<bos>', 'i', '<unk>', '<unk>', 'and', 'i', '<unk>', '<unk>', '<unk>', '.', '<eos>']


In [70]:
print(en_vocab.get_index("i"))

14


We create a numericalize function which we'll to "numericalize" (a fancy way of saying convert tokens to indices) our tokens in each example using the vocabularies and return the result into new "en_ids" and "vn_ids" features.

In [99]:
def numericalize(data):
    dict = {"en_ids": [], "vi_ids": []}
    for i in range(len(data["en_tokens"])):
        en_ids = en_vocab.encode(data["en_tokens"][i])
        vi_ids = vi_vocab.encode(data["vi_tokens"][i])
        dict["en_ids"].append(en_ids)
        dict["vi_ids"].append(vi_ids)
    return dict

Now, we apply the numericalize_example function to the dataset

In [102]:
train_data_numericalized = numericalize(train_data_tokenized)
test_data_numericalized = numericalize(test_data_tokenized)
valid_data_numericalized = numericalize(valid_data_tokenized)

Checking an example, we can see that it has the two new features: "en_ids" and "de_ids", both a list of integers representing their indices in the respective vocabulary.

In [105]:
list(train_data_numericalized.values())[0][0]

[4, 70, 24, 71, 16, 72, 5, 3]

We can confirm the indices are correct by using the decode method with the corresponding vocabulary on the list of indices.

In [106]:
en_vocab.decode(list(train_data_numericalized.values())[0][0])

['<sos>', 'there', 'are', 'thousands', 'of', 'glens', '.', '<eos>']

The indices in the example above are Python integers. They need to be converted to PyTorch tensors in order to use them with PyTorch. Therefore, we will convert them to PyTorch tensors.

In [115]:
def convert_to_tensors(data):
    dict = {"en_ids": [], "vi_ids": []}
    for i in range(len(data["en_ids"])):
        en_ids = torch.tensor(data["en_ids"][i])
        vi_ids = torch.tensor(data["vi_ids"][i])
        dict["en_ids"].append(en_ids)
        dict["vi_ids"].append(vi_ids)
    return dict

In [116]:
train_data_numericalized = convert_to_tensors(train_data_numericalized)
test_data_numericalized = convert_to_tensors(test_data_numericalized)
valid_data_numericalized = convert_to_tensors(valid_data_numericalized)

We can confirm this worked by checking an example and seeing the "en_ids" and "de_ids" features are listed as tensor([...]).

In [117]:
train_data_numericalized

{'en_ids': [tensor([ 4, 70, 24, 71, 16, 72,  5,  3]),
  tensor([ 4, 14, 73,  8, 15, 21,  5,  3]),
  tensor([ 4, 34, 44, 74, 75, 76, 77, 45, 78,  5,  3]),
  tensor([ 4, 14, 79, 80,  8, 81,  9, 82,  5,  3]),
  tensor([ 4, 25, 26, 83,  8, 46, 47, 35, 84, 48, 10,  3]),
  tensor([ 4, 85,  6, 36, 12, 86, 87, 88, 89, 27, 90, 91,  5,  3]),
  tensor([ 4, 17, 14, 49, 37, 22, 92,  5,  3]),
  tensor([ 4, 93, 50,  7, 11, 94,  6, 95, 18, 10,  3]),
  tensor([  4,  18,  12,  28,  96,  35,  11,  51,  16,  97,   7,  98,  29,  99,
          100,  52,  11,  51,  16, 101,   5,   3]),
  tensor([  4,  18,  12,  38, 102,  30,  15, 103,  16,  23, 104,  39,  13,  39,
            5,   3]),
  tensor([  4,  40,   6,  20,   6, 105,  20,  36,  41, 106,   6, 107,   5,   3]),
  tensor([  4,  38,  30,   5, 108,  48, 109,   6, 110,  12, 111,  31,  32,  53,
          112,  13,  29, 113,  31, 114,  29,  54,  15, 115,   5,   3]),
  tensor([  4,  15,   9, 116,   6,  20, 117, 118,   9, 119,  55,   7, 120, 121,
          122,

## Data Loaders

The final step of preparing the data is to create the data loaders. These can be iterated upon to return a batch of data, each batch being a dictionary containing the numericalized English and Vietnamese sentences (which have also been padded) as PyTorch tensors.

First, we need to create a function that collates, i.e. combines, a batch of examples into a batch. The collate_fn below takes a "batch" as input (a list of examples), we then separate out the English and German indices for each example in the batch, and pass each one to the pad_sequence function. pad_sequence takes a list of tensors, pads each one to the length of the longest tensor using the padding_value (which we set to pad_index, the index of our <pad> token) and then returns a [max length, batch size] shaped tensor, where batch size is the number of examples in the batch and max length is the length of the longest tensor in the batch. We put each tensor into a dictionary and then return it.

The get_collate_fn takes in the padding token index and returns the collate_fn defined inside it. This technique, of defining a function inside another and returning it, is known as a closure. It allows the collate_fn to continually use the value of pad_index it was created with without creating a class or using global variables.

In [118]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

Next, we write the functions which give us our data loaders creating using PyTorch's DataLoader class.

get_data_loader is created using a Dataset, the batch size, the padding token index (which is used for creating the batches in the collate_fn, and a boolean deciding if the examples should be shuffled at the time the data loader is iterated over.

The batch size defines the maximum amount of examples within a batch. If the length of the dataset is not evenly divisible by the batch size then the last batch will be smaller.

In [119]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

Finally, we create our data loaders.

To reduce the training time, we generally want to us the largest batch size possible. When using a GPU, this means using the largest batch size that will fit in GPU memory.

Shuffling of data makes training more stable and potentially improves the final performance of the model, however only needs to be done on the training set. The metrics calculated for the validation and test set will be the same no matter what order the data is in.

In [122]:
batch_size = 128
pad_index = en_vocab.get_index('<pad>')

train_data_loader = get_data_loader(train_data_numericalized, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data_numericalized, batch_size, pad_index)
test_data_loader = get_data_loader(test_data_numericalized, batch_size, pad_index)

In [123]:
train_data_loader